In [3]:
import numpy as np
from PIL import Image

# 1. Collect dataset
&emsp;&emsp; We collect different faces of different cubes in different light colors(ambient) shot by different cameras. 
To make a dataset easily, we shoot video. In each video, the cube, the face color, the light color and the camera hold same. 
We only change light intensity. After shooting videos, we tag their labels. Then, we begin make dataset of the cubes.

## 1.1 Clipping videos
&emsp;&emsp; Clip the videos in /video directory

In [ ]:
import cv2
import numpy as np
import os
from multiprocessing import Pool

v_ext_names = ['mov', 'mp4', 'avi']

def read_images(video_file_name):
    video = cv2.VideoCapture(video_file_name)
    frames = []
    while True:
        success, frame = video.read()
        if not success:
            break
        frames.append(frame)
    return np.stack(frames, axis=0)

def handle_video(v_name):
    v_path = os.path.join('video', v_name)
    pic_path = os.path.join('video_frames', v_name)
    frames = read_images(v_path)
    frames_count = frames.shape[0]
    for i in np.arange(frames_count):
        frame = frames[i]
        cv2.imwrite(os.path.join(pic_path, str(i) + '.png'), frame)

pool = Pool(processes=8)
video_names = list(filter(lambda x: ('.' in x) and (x.split('.')[-1].lower() in v_ext_names) , os.listdir('./video')))
for v_name in video_names:
    pool.apply_async(handle_video, args=(v_name,))
pool.close()
pool.join()
        

## 1.2 Rubik's cube separation
&emsp;&emsp; Separate rubik cube in the pictures and generate labels in 'label' directory.

In [2]:
import cv2
import numpy as np
import pandas as pd
import os
from multiprocessing import Pool

def clip_one_picture(picture_file, ux, uy, dx, dy, output_file):
    img = cv2.imread(picture_file)
    img = img[ux: dx + 1, uy: dy + 1, :]
    cv2.imwrite(output_file, img)

def list_images(dir_name):
    img_suffix = ['png', 'jpg']
    images = filter(lambda x: '.' in x and x.split('.')[-1].lower() in img_suffix, os.listdir(dir_name))
    return list(images)

def clip_multiple_pictures(input_file_names, ux, uy, dx, dy, output_dir_name, start_number):
    idx = start_number
    for img_file_name in input_file_names:
        img_out_name0 = os.path.join(output_dir_name, str(idx)) + '.png'
        clip_one_picture(img_file_name, ux, uy, dx, dy, img_out_name0)
        idx += 1
    
print("Please wait")

video_dir_names = filter(lambda x: '.' in x and x.split('.')[-1].lower() == 'mov', os.listdir('video_frames'))
tb_video = pd.read_csv('./label/label_video.tsv', sep='\t', index_col='id',
                       dtype={'id': int})
# tb_picture = pd.DataFrame(columns=['id', 'camera', 'light_color', 'cube_id', 'cube_color', 
#                                    'video_file_name', 'picture_file_name'], index=['id'])
tb_picture = []
pool = Pool(8)

dict_video_file_name_to_info = dict()
columns = ['camera', 'light_color', 'cube_id', 'cube_color', 'file_name', 'ux', 'uy', 'dx', 'dy']
ids = list(tb_video.index)
for i in range(len(tb_video)):
    file_name = tb_video.loc[ids[i], 'file_name']
    dict_video_file_name_to_info[file_name] = {}
    for c in columns:
        dict_video_file_name_to_info[file_name][c] = tb_video.loc[ids[i], c]

MULTITHREAD = True
i = 0
for video_dir_name in video_dir_names:
    video_dir_name0 = os.path.join('video_frames', video_dir_name)
    image_file_mames = list_images(video_dir_name0)
    input_file_names = [os.path.join(video_dir_name0, name) for name in image_file_mames]
    endi = i + len(image_file_mames)
    
    if video_dir_name not in dict_video_file_name_to_info:  # some videos in video_frames directory may not have been labeled
        continue
    info = dict_video_file_name_to_info[video_dir_name]
    if MULTITHREAD:
        pool.apply_async(clip_multiple_pictures, args=(input_file_names, info['ux'], info['uy'], 
                                                       info['dx'], info['dy'], 'clipped_video_frames', i))
    else:
        clip_multiple_pictures(input_file_names, info['ux'], info['uy'], info['dx'], info['dy'], 
                               'clipped_video_frames', i)
    for j in np.arange(i, endi):
        pic_info = info.copy()
        pic_info.pop('file_name')
        pic_info.pop('ux')
        pic_info.pop('uy')
        pic_info.pop('dx')
        pic_info.pop('dy')
        pic_info['video_file_name'] = info['file_name']
        pic_info['picture_file_name'] = str(j) + '.png'
        pic_info['id'] = j
        tb_picture.append(pic_info)
    i = endi

tb_picture = pd.DataFrame.from_records(tb_picture, index=['id'])

pool.close()
pool.join()
tb_picture.to_csv('./label/label_picture.tsv', sep='\t')
print("Finish")

Please wait
Finish


# 1.3 Rubik's cube color block separation
&emsp;&emsp; In this subsection, we separate color blocks in the rubik cube. And by the way generate the label of those 
color blocks. Hence, we will have a block dataset of the rubik's cube.

In [3]:
import os
import cv2
import pandas as pd
from multiprocessing import Pool

def cut3x3(picture_file, output_directory, start_number):
    img = cv2.imread(picture_file)
    h, w, c = img.shape
    h0 = h // 3
    w0 = w // 3
    k = start_number
    for i in range(3):
        for j in range(3):
            img2 = img[i * h0: (i + 1) * h0, j * w0: (j + 1) * w0, :]
            cv2.imwrite(os.path.join(output_directory, str(k)) + '.png', img2)
            k += 1

label_pictures = pd.read_csv('./label/label_picture.tsv', sep='\t', index_col='id')

label_blocks = []
columns = label_pictures
ids = label_pictures.index

pool = Pool(8)

position_strings = ['UL', 'UM', 'UR', 'ML', 'MM', 'MR', 'DL', 'DM', 'DR']

k = 0
MULTITHREAD = True
for i in range(len(label_pictures)):
    file_name = label_pictures.loc[ids[i], 'picture_file_name']
    picture_file = os.path.join('./clipped_video_frames', file_name)
    if MULTITHREAD:
        pool.apply_async(cut3x3, args=(picture_file, './clipped_color_blocks', k))
    else:
        cut3x3(picture_file, './clipped_color_blocks', k)
    
    record = label_pictures.loc[ids[i]].to_dict()
    for j in range(9):
        record2 = record.copy()
        record2['block_file_name'] = str(k + j) + '.png'
        record2['block_position'] = position_strings[j]
        record2['id'] = k + j
        label_blocks.append(record2)    
    k += 9

label_blocks = pd.DataFrame.from_records(label_blocks, index=['id'])
label_blocks.to_csv('./label/label_block.tsv', sep='\t')
pool.close()
pool.join()

print("Finish")

Finish


# 2. Plot the histograms of different cubes

 &emsp;&emsp; In colordetect/data folder, we have examples of red cube, blue cube, and green cube. In each of these directory, 
 we present different cubes.  
 &emsp;&emsp; After that, we write functions to identify color.
 

# 3. Detect the color sequence of the cube, given an image array that has been selected.

In [ ]:
def detect_color(img_arr):
    c, w, h = img_arr.shape
    
